In [1]:
import pandas as pd
import numpy as np
from kaggle_handler import handler

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

# Downloading/Loading Dataset

In [3]:
Assets = handler('yasserh/titanic-dataset')

Directory 'Assets' already exists.
Datasets already exist in Assets folder
['Titanic-Dataset.csv']
 Change Add_more parameter to download more datasets


In [4]:
Assets

['Titanic-Dataset.csv']

In [5]:
df = pd.read_csv('Assets/Titanic-Dataset.csv',
                usecols=['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch','Fare', 'Embarked'])
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# Spliting Dataset

In [6]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=.2)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((712, 7), (712,), (179, 7), (179,))

In [7]:
X_train.isna().sum()

Pclass        0
Sex           0
Age         133
SibSp         0
Parch         0
Fare          0
Embarked      1
dtype: int64

In [8]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
538,3,male,NaN,0,0,14.5,S
466,2,male,NaN,0,0,0.0,S


# Column Transformer

In [9]:
trf1 = ColumnTransformer(transformers=[('si_age',SimpleImputer(strategy='mean'),[2]),
                                     ('si_embarked',SimpleImputer(strategy="most_frequent"),[6])
                                    ],
                      remainder='passthrough')

In [10]:
trf2 = ColumnTransformer(transformers=[('ohe_sex_embarked',OneHotEncoder(sparse_output=False,
                                                                         dtype=np.int32,
                                                                         handle_unknown='ignore'),[3,1])],
                         remainder='passthrough')

In [11]:
trf3 = ColumnTransformer(transformers=[('scale',MinMaxScaler(),slice(0,10))],
                         remainder='passthrough')

In [12]:
trf4 = SelectKBest(score_func=chi2, k=10)

# Model

In [13]:
dtc = DecisionTreeClassifier(max_depth=3)

# Pipeline

In [14]:
pipe = Pipeline([('trf1',trf1),
                 ('trf2',trf2),
                 ('trf3',trf3),
                 ('trf4',trf4),
                 ('dtc',dtc)])

# Treaning Model

In [15]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('si_age', SimpleImputer(),
                                                  [2]),
                                                 ('si_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [3, 1])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(score_func=<function chi2 at 0x7fd6800279c0>)),
                ('dtc', DecisionTreeClassifier(max_depth=3))])

# Pipeline Peramiters

In [16]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('si_age', SimpleImputer(), [2]),
                                 ('si_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                handle_unknown='ignore',
                                                sparse_output=False),
                                  [3, 1])]),
 'trf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(score_func=<function chi2 at 0x7fd6800279c0>),
 'dtc': DecisionTreeClassifier(max_depth=3)}

In [17]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('si_age', SimpleImputer(), [2]),
                                ('si_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [18]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.79131261])

In [19]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [20]:
y_pred = pipe.predict(X_test)
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0])

# Accuracy Score

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8603351955307262

# Cross Validation

In [22]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=100, scoring='accuracy').mean()

np.float64(0.8150000000000002)

# GridSearch using Pipeline

In [23]:
params = {
    'dtc__max_depth':[1,2,3,4,5,None]
}

In [24]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=100, scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=100,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('si_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('si_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [3,
                                                                          1])])),
                                       ('trf3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(score_func=<function chi2 at 0x7fd6800279c0>)),
                                       ('dtc',
                                        DecisionTreeClassifier(max_depth=3))]),
             param_grid={'dtc__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [25]:
grid.best_score_

np.float64(0.8150000000000002)

In [26]:
grid.best_params_

{'dtc__max_depth': 3}

# Saving Model

In [27]:
import pickle
import os

if 'with_pipline_models' not in os.listdir():
    os.mkdir('with_pipline_models')

pickle.dump(pipe, open('with_pipline_models/model.pkl','wb'))